In [73]:
import pandas as pd
import numpy as np

In [95]:
data21 = pd.read_csv("College_Scorecard_Raw_Data_09242024/FieldOfStudyData1718_1819_PP.csv")
data21 = data21[['UNITID','INSTNM', 'CIPDESC', 'CREDDESC', 'CIPCODE', 'CONTROL', 'EARN_MALE_NE_MDN_3YR', 'EARN_NOMALE_NE_MDN_3YR', 'DEBT_MALE_STGP_EVAL_MEAN', 'DEBT_NOTMALE_STGP_EVAL_MDN']]
# data21[data21['EARN_MALE_NE_MDN_3YR'].notna()]
# data21['EARN_MALE_NE_MDN_3YR'].value_counts()


In [106]:
class makeDataSet:
    def __init__(self):
        self.dataFrames = []
        self.reducedDataFrames = []
        self.allColumns = []
        # file name are any one revicing a degree in the phscal year
        self.yearFiles = ["1415_1516_PP.csv", "1516_1617_PP.csv", "1617_1718_PP.csv", "1718_1819_PP.csv", "1819_1920_PP.csv", "1920_2021_PP.csv"]
        self.endYears = ["2016", "2017", "2018", "2019", "2020", "2021"]
        
    def readInDataSets(self):
        basePath = "College_Scorecard_Raw_Data_09242024/FieldOfStudyData"
        i = 0
        for year in self.yearFiles:
            print(self.endYears[i])
            data = pd.read_csv(basePath+year)
            data["year"] = self.endYears[i]
            self.dataFrames.append(data)
            i+=1
            
    def mergeYears(self):
        self.finalDataSet = pd.concat(self.dataFrames)
        print(self.finalDataSet.year.unique())
        self.finalDataSet.dropna()
        print("merge years")
        print(self.finalDataSet.year.unique())
#         d.finalDataSet['UNITID'] = d.finalDataSet['UNITID'].astype(int)
    
    def makeDataSetTotal(self):
        data = self.finalDataSet[['UNITID', 'INSTNM', 'year', 'CIPDESC', 'CREDDESC', 'CIPCODE', 'CONTROL', 'EARN_NE_MDN_3YR', 'DEBT_ALL_STGP_EVAL_MDN']]
        data = data.dropna()
        data = data[data.EARN_NE_MDN_3YR != 'PrivacySuppressed']
        data = data[data.DEBT_ALL_STGP_EVAL_MDN != 'PrivacySuppressed']
        data['UNITID'] = data['UNITID'].astype(int)
        data = pd.merge(data, self.Ipeds, on='UNITID', how='inner')
#         data = data[data['CREDDESC']=='Bachelor’s Degree']
        return data
    
    def makePellDataSet(self):
        data = self.finalDataSet[['UNITID','INSTNM', 'year', 'CIPDESC', 'CREDDESC', 'CIPCODE', 'CONTROL','EARN_PELL_NE_MDN_3YR','EARN_NOPELL_NE_MDN_3YR','DEBT_PELL_STGP_EVAL_MDN','DEBT_NOPELL_STGP_EVAL_MDN']]
        data = data.dropna()
        data = data[data.EARN_PELL_NE_MDN_3YR != 'PrivacySuppressed']
        data = data[data.EARN_NOPELL_NE_MDN_3YR != 'PrivacySuppressed']
        data = data[data.DEBT_PELL_STGP_EVAL_MDN != 'PrivacySuppressed']
        data = data[data.DEBT_NOPELL_STGP_EVAL_MDN != 'PrivacySuppressed']
        data = data[data.EARN_PELL_NE_MDN_3YR != 'PS']
        data = data[data.EARN_NOPELL_NE_MDN_3YR != 'PS']
        data = data[data.DEBT_PELL_STGP_EVAL_MDN != 'PS']
        data = data[data.DEBT_NOPELL_STGP_EVAL_MDN != 'PS']
        data['UNITID'] = data['UNITID'].astype(int)
        data = pd.merge(data, self.Ipeds, on='UNITID', how='inner')
        data['no_pell_diff'] = data['EARN_NOPELL_NE_MDN_3YR'].astype(int)-data['EARN_PELL_NE_MDN_3YR'].astype(int)
        return data
    
    def makeMaleDataSet(self):
        print(self.finalDataSet.year.unique())
        data = self.finalDataSet[['UNITID','INSTNM', 'year', 'CIPDESC', 'CREDDESC', 'CIPCODE', 'CONTROL', 'EARN_MALE_NE_MDN_3YR', 'EARN_NOMALE_NE_MDN_3YR', 'DEBT_MALE_STGP_EVAL_MEAN', 'DEBT_NOTMALE_STGP_EVAL_MDN']]
        print(data.year.unique())
        data = data.dropna()
        print(data.year.unique())
        data = data[data.EARN_MALE_NE_MDN_3YR != 'PrivacySuppressed']
        data = data[data.EARN_NOMALE_NE_MDN_3YR != 'PrivacySuppressed']
        data = data[data.DEBT_MALE_STGP_EVAL_MEAN != 'PrivacySuppressed']
        data = data[data.DEBT_NOTMALE_STGP_EVAL_MDN != 'PrivacySuppressed']
        data = data[data.EARN_MALE_NE_MDN_3YR != 'PS']
        data = data[data.EARN_NOMALE_NE_MDN_3YR != 'PS']
        data = data[data.DEBT_MALE_STGP_EVAL_MEAN != 'PS']
        data = data[data.DEBT_NOTMALE_STGP_EVAL_MDN != 'PS']
        data['UNITID'] = data['UNITID'].astype(int)
        print(data.year.unique())
        data = pd.merge(data, self.Ipeds, on='UNITID', how='inner')
        data['male_differential'] = data['EARN_MALE_NE_MDN_3YR'].astype(int)-data['EARN_NOMALE_NE_MDN_3YR'].astype(int)
        return data
    
    def readInIpeds(self):
        self.Ipeds = pd.read_csv("ipeds_2019/CSV_322025-519/CSV_322025-519.csv")
        self.Ipeds['UNITID'] = self.Ipeds['unitid']
        self.Ipeds.drop(columns=['unitid'])
        

        
        

In [107]:
d = makeDataSet()
d.readInDataSets()
d.mergeYears()
d.readInIpeds()
# pell/no pell data set
dataSetPell = d.makePellDataSet()
# women/man data set
dataSetMale = d.makeMaleDataSet()

2016
2017
2018
2019
2020
2021
['2016' '2017' '2018' '2019' '2020' '2021']
merge years
['2016' '2017' '2018' '2019' '2020' '2021']
['2016' '2017' '2018' '2019' '2020' '2021']
['2016' '2017' '2018' '2019' '2020' '2021']
['2018' '2019']
['2019']


In [108]:
pd.set_option('display.max_rows', 100)
nan_counts = dataSetMale.isna().sum()
df_filtered = dataSetMale.loc[:, nan_counts <= 10]
df_filtered.to_csv("male_femal_dataset.csv")

In [109]:
df_filtered

,UNITID,INSTNM,year_x,CIPDESC,CREDDESC,CIPCODE,CONTROL,EARN_MALE_NE_MDN_3YR,EARN_NOMALE_NE_MDN_3YR,DEBT_MALE_STGP_EVAL_MEAN,...,HD2019.Postsecondary and Title IV institution indicator,HD2019.Institutional category,HD2019.Carnegie Classification 2018: Basic,HD2019.Carnegie Classification 2018: Undergraduate Instructional Program,HD2019.Carnegie Classification 2018: Graduate Instructional Program,HD2019.Carnegie Classification 2018: Undergraduate Profile,HD2019.Carnegie Classification 2018: Enrollment Profile,HD2019.Carnegie Classification 2018: Size and Setting,HD2019.Institution size category.1,male_differential
0,100663,University of Alabama at Birmingham,2019,Communication and Media Studies.,Bachelor’s Degree,901,Public,34592,34576,20425,...,Title IV postsecondary institution,"Degree-granting, primarily baccalaureate or above",Doctoral Universities: Very High Research Acti...,"Professions plus arts & sciences, high graduat...",Research Doctoral: STEM-dominant,"Four-year, medium full-time , selective, highe...",Majority undergraduate,"Four-year, large, primarily nonresidential","20,000 and above",16
1,100663,University of Alabama at Birmingham,2019,"Engineering, General.",Master's Degree,1401,Public,90431,98288,32844,...,Title IV postsecondary institution,"Degree-granting, primarily baccalaureate or above",Doctoral Universities: Very High Research Acti...,"Professions plus arts & sciences, high graduat...",Research Doctoral: STEM-dominant,"Four-year, medium full-time , selective, highe...",Majority undergraduate,"Four-year, large, primarily nonresidential","20,000 and above",-7857
2,100663,University of Alabama at Birmingham,2019,"Psychology, General.",Bachelor’s Degree,4201,Public,27501,33227,28687,...,Title IV postsecondary institution,"Degree-granting, primarily baccalaureate or above",Doctoral Universities: Very High Research Acti...,"Professions plus arts & sciences, high graduat...",Research Doctoral: STEM-dominant,"Four-year, medium full-time , selective, highe...",Majority undergraduate,"Four-year, large, primarily nonresidential","20,000 and above",-5726
3,100663,University of Alabama at Birmingham,2019,Criminal Justice and Corrections.,Bachelor’s Degree,4301,Public,43827,38520,24016,...,Title IV postsecondary institution,"Degree-granting, primarily baccalaureate or above",Doctoral Universities: Very High Research Acti...,"Professions plus arts & sciences, high graduat...",Research Doctoral: STEM-dominant,"Four-year, medium full-time , selective, highe...",Majority undergraduate,"Four-year, large, primarily nonresidential","20,000 and above",5307
4,100663,University of Alabama at Birmingham,2019,Dentistry.,First Professional Degree,5104,Public,141123,139659,170550,...,Title IV postsecondary institution,"Degree-granting, primarily baccalaureate or above",Doctoral Universities: Very High Research Acti...,"Professions plus arts & sciences, high graduat...",Research Doctoral: STEM-dominant,"Four-year, medium full-time , selective, highe...",Majority undergraduate,"Four-year, large, primarily nonresidential","20,000 and above",1464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10031,494588,Pima Medical Institute-San Antonio,2019,Health and Medical Administrative Services.,Undergraduate Certificate or Diploma,5107,"Private, for-profit",34850,26527,7874,...,Title IV postsecondary institution,"Nondegree-granting, sub-baccalaureate","Not applicable, not in Carnegie universe (not ...","Not applicable, not in Carnegie universe (not ...","Not applicable, not in Carnegie universe (not ...","Not applicable, not in Carnegie universe (not ...","Not applicable, not in Carnegie universe (not ...","Not applicable, not in Carnegie universe (not ...","Under 1,000",8323
10032,494588,Pima Medical Institute-San Antonio,2019,Allied Health and Medical Assisting Services.,Undergraduate Certificate or Diploma,5108,"Private, for-profit",30328,28089,7599,...,Title IV postsecondary institution,"Nondegree-g

In [110]:
pd.set_option('display.max_rows', 100)
nan_counts2 = dataSetPell.isna().sum()
df_filtered2 = dataSetPell.loc[:, nan_counts2 <= 10]
df_filtered2.to_csv("pell_dataset.csv")

Figuring out why only has 2019 in data set (nan for all other years)

In [43]:
data = d.finalDataSet
data = data[['UNITID','INSTNM', 'year', 'CIPDESC', 'CREDDESC', 'CIPCODE', 'CONTROL', 'EARN_MALE_NE_MDN_3YR', 'EARN_NOMALE_NE_MDN_3YR']]#, 'DEBT_MALE_STGP_EVAL_MEAN', 'DEBT_NOTMALE_STGP_EVAL_MDN']]
data = data.dropna()
data = data[data.EARN_MALE_NE_MDN_3YR != 'PrivacySuppressed']
data = data[data.EARN_NOMALE_NE_MDN_3YR != 'PrivacySuppressed']
# data = data[data.DEBT_MALE_STGP_EVAL_MEAN != 'PrivacySuppressed']
# data = data[data.DEBT_NOTMALE_STGP_EVAL_MDN != 'PrivacySuppressed']
data = data[data.EARN_MALE_NE_MDN_3YR != 'PS']
data = data[data.EARN_NOMALE_NE_MDN_3YR != 'PS']
# data = data[data.DEBT_MALE_STGP_EVAL_MEAN != 'PS']
# data = data[data.DEBT_NOTMALE_STGP_EVAL_MDN != 'PS']

data

,UNITID,INSTNM,year,CIPDESC,CREDDESC,CIPCODE,CONTROL,EARN_MALE_NE_MDN_3YR,EARN_NOMALE_NE_MDN_3YR
55,100654.0,Alabama A & M University,2019,"Business Administration, Management and Operat...",Bachelor’s Degree,5202,Public,50924,37080
56,100654.0,Alabama A & M University,2019,"Business Administration, Management and Operat...",Master's Degree,5202,Public,51692,41403
62,100663.0,University of Alabama at Birmingham,2019,Communication and Media Studies.,Bachelor’s Degree,901,Public,34592,34576
73,100663.0,University of Alabama at Birmingham,2019,Educational Administration and Supervision.,Master's Degree,1304,Public,61537,54559
89,100663.0,University of Alabama at Birmingham,2019,"Engineering, General.",Master's Degree,1401,Public,90431,98288
...,...,...,...,...,...,...,...,...,...
218758,494588.0,Pima Medical Institute-San Antonio,2019,Clinical/Medical Laboratory Science/Research a...,Undergraduate Certificate or Diploma,5110,"Private, for-profit",33128,25623
218784,494746.0,North-West College-San Diego,2019,"Practical Nursing, Vocational Nursing and Nurs...",Undergraduate Certificate or Diploma,5139,"Private, for-profit",50092,43129
218800,494843.0,Fortis College-Landover,2019,Dental Support Services and Allied Professions.,Undergraduate Certificate or Diploma,5106,"Private, for-profit",29915,27533
218803,494843.0,Fortis College-Landover,2019,Allied Health and Medical Assisting Services.,Undergraduate Certificate or Diploma,5108,"Private, for-profit",23781,26739
